# pandas的stack和pivot实现数据透视
<font size="5" color="#dd0000">**将劣势数据变成二位交叉形式, 便于分析, 叫做重塑或透视**</font>
<img src="./image/stack_pivot.png">

1. 经过统计的到多为指标数据
2. 使用unstack实现数据二维透视
3. 使用pivot简化透视
4. stack、unstack、pivot的语法

**1.经过统计得到多为度指标数据**
非常场景的统计场景, 指定多个维度, 计算聚合后的指标
示例: 统计的到点光源评分数据集, 每个月份的每个分数被评多份多少次(月份、分数1~5、次数)

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv('./data/movies/ratings.csv')

In [ ]:
df.head()

In [ ]:
# pandas处理日期
df['pddate'] = pd.to_datetime(df['timestamp'], unit='s')

In [ ]:
df.head()

In [ ]:
df.dtypes

**注意: 使用pd.to_datetime()函数所转换的时间戳可以直接使用pandas中的日期函数, 列如:**
1. dt.month
2. dt.day

In [ ]:
df['pddate'].dt.month

In [ ]:
# 实现数据统计
df_group = df.groupby([df['pddate'].dt.month, 'rating'])

In [ ]:
df_group

In [ ]:
# 取分组数据
df_group.get_group((4, 0.5))

In [ ]:
df_group = df_group['userId'].agg(pv=np.size)

In [ ]:
df_group.head(20)

这样的数据格式, 想查看按月份, 不同评分次数趋势, 没办法实现
需要将数据变成每个评分是一列才可以实现

**2.使用unstack实现数据二维透视**
目的: 想要画图对比安装月份的不同评分的数量趋势

In [ ]:
df_stack = df_group.unstack()
df_stack

In [ ]:
# 查看某一个pv的数据
df_stack[('pv', 1.0)]

In [ ]:
# 查看指定月份和指定pv数据
df_stack.loc[1, ('pv', 2.0)]

In [ ]:
df_stack.plot()

In [ ]:
#unstack和stack互逆操作
df_stack.stack().head(20)

## 3.使用pivot简化透视

In [ ]:
df_group.head(20)

In [ ]:
df_reset = df_group.reset_index()
df_reset.head()

In [ ]:
df_pivot = df_reset.pivot(index='pddate', columns='rating', values='pv')

In [ ]:
df_pivot.head()

In [ ]:
df_pivot.plot()

**pivot方法相当于对DataFrame使用set_index创建分层索引, 然后调用unstack**


## 4. stack、unstack、pivot的语法

**stack:DataFrame.stack(level=-1, dropna=True), 将column变成index, 类似把横放的书记变成竖放**
level=-1代表多层索引的最内层, 可以通过level=0,1,2指定多层索引的对应层

<img src="./image/stack.png">

**unstack:DataFrame.unstack(level=-1, fill_value=None), 将index变成column, 类似把竖放的书变为横放**
<img src="./image/unstack.png">

**pivot:DataFrame.pivot(index=None, columns=None, values=None), 指定index, columns, values实现二维透视**
<img src="./image/pivot.png">